In [2]:
import numpy as np

# Pseudocódigo de predição

```
predict(x)
    y_chapeu = self.w.x + self.bias
    return self.activation(y_chapeu),y_chapeu
    
self.activation = signal
```

# Pseudocódigo da signal

```
signal(value)
    boolean_value = value>0
    return 2*boolean_value-1
```

# Pseudocódigo do custo

```
custo(predição,y_chapeu,y)
    is_error = (predição-y)/2
    return is_error*(y_chapeu-y)
```

# Pseudocódigo do treinamento

```
fit(x,y,max)
    error <- 1
    count <- 0
    while error > 0 and count < max
        error <- 0
        count <- count + 1
        for (_x,_y) in dataset
            predição, y_chapeu <- self.predict(x)
            local_error <- self.cost(predição,y_chapeu,y)
            self.update_weights(x,y,local_error)
            error <- error + abs(local_error)
```

# Pseudocódigo de atualização de pesos

```
update_weights(x,y,local_error)
    self.w <- self.w + x*y*local_error
    self.bias <- self.bias + y*local_error
```

Onde:

* $W$ é o vetor de pesos.

* $bias$ é o bias.

* $x$ é o vetor de entrada

* $y$ é o valor esperado

* $\hat{y}$ é o valor calculado

In [3]:
def signal(value):
    boolean_value = value > 0
    return 2*boolean_value-1

class Perceptron(object):
    
    def __init__(self,x_example,activation,initial_value=0):
        self.W = np.zeros_like(x_example) + initial_value
        self.bias = initial_value
        self.activation = activation
        
    def predict(self,x):
        y_chapeu = self.W.dot(x) + self.bias
        return self.activation(y_chapeu), y_chapeu

    def update_weights(self,x,y,local_error):
        self.bias += y*local_error
        self.W += y*x*local_error

    def cost(self, predição, y_chapeu,y):
        is_error = (predição-y)/2
        return is_error*(y_chapeu-y)
            
    def fit(self,x,y,_max=1000):
        error = 1
        count = 0
        while error > 0 and count < _max:
            error = 0
            count += 1
            for _x,_y in zip(x,y):
                predição, y_chapeu = self.predict(_x)
                local_error = self.cost(predição,y_chapeu,_y)
                self.update_weights(_x,_y,local_error)
                error = error + abs(local_error)
        return error == 0, count

# Criação dos datasets

Dataset binário para AND, OR e XOR

In [4]:
possible_x = np.array([[0,0],[0,1],[1,0],[1,1]],dtype=np.float64)

dataset_and = dict(x=possible_x,y=np.array([-1,-1,-1,1]))
dataset_or = dict(x=possible_x,y=np.array([-1,1,1,1]))
dataset_xor = dict(x=possible_x,y=np.array([-1,1,1,-1]))

# Teste para o and

In [5]:
and_perceptron = Perceptron(dataset_and["x"][0],signal)
result = and_perceptron.fit(**dataset_and)
result, and_perceptron.W, and_perceptron.bias

((True, 3), array([ 1.,  1.]), -1.0)

# Teste para o or

In [6]:
or_perceptron = Perceptron(dataset_or["x"][0],signal)
result = or_perceptron.fit(**dataset_or)
result, or_perceptron.W, or_perceptron.bias

((True, 4), array([ 2.,  2.]), 0.0)

# Teste para o xor

In [7]:
xor_perceptron = Perceptron(dataset_xor["x"][0],signal)
result = xor_perceptron.fit(**dataset_xor)
result, xor_perceptron.W, xor_perceptron.bias

((False, 1000), array([-4001., -2000.]), 1996.0)

O xor é impossível com os dados atuais pois temos uma impossibilidade lógica

<center> $x = [0,0]$ $y = -1$. <b>Logo</b> $bias \leq 0$</center>
<center> $x = [1,0]$ $y = 1$. <b>Logo</b> $-bias \leq W[0] $</center>
<center> $x = [0,1]$ $y = 1$. <b>Logo</b> $-bias \leq W[1] $</center>
<center> $x = [1,1]$ $y = -1$. <b>Logo</b> $W[0] + W[1] \leq -bias$</center>

<center>$bias \leq 0 \implies |bias| = - bias$  </center>

<center>Então

$$ |bias| \leq W[0] \implies W[0] \geq 0 $$
$$ |bias| \leq W[1] \implies W[1] \geq 0$$
$$ W[0] + W[1] \leq |bias|$$

logo

$$ W[0] \geq |bias| \land W[1] \geq |bias| \land W[0] + W[1] \leq |bias| $$

<h1>Impossível</h1>
</center>

## Solução?

Para resolvermos esse problema vamos adicionar um valor para a entrada que é o valor do and entre os dois valores de entrada. Feito isso teremos:

<center> $x = [0,0,0]$ $y = -1$. <b>Logo</b> $bias \leq 0$</center>
<center> $x = [1,0,0]$ $y = 1$. <b>Logo</b> $-bias \leq W[0] $</center>
<center> $x = [0,1,0]$ $y = 1$. <b>Logo</b> $-bias \leq W[1] $</center>
<center> $x = [1,1,1]$ $y = -1$. <b>Logo</b> $W[0] + W[1] + W[2] \leq -bias$</center>

<center>$bias \leq 0 \implies |bias| = - bias$  </center>

<center>Então

$$ |bias| \leq W[0] \implies W[0] \geq 0 $$
$$ |bias| \leq W[1] \implies W[1] \geq 0$$
$$ W[0] + W[1] + W[2] \leq |bias|$$

logo

$$ W[0] \geq |bias| \land W[1] \geq |bias| \land W[2] \leq |bias| - |W[0]| - |W[1]| $$

<h1>Não impossível</h1>
</center>

In [8]:
dataset_xor = dict(x=np.array(
    [[0,0,0],[0,1,0],[1,0,0],[1,1,1]],dtype=np.float64),
                   y=dataset_xor["y"])

xor_perceptron = Perceptron(dataset_xor["x"][0],signal)
result = xor_perceptron.fit(**dataset_xor)
result, xor_perceptron.W, xor_perceptron.bias

((True, 7), array([ 2.,  2., -9.]), -1.0)